# Cell type classification for LN dataset

2024-06-05

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans

In [ ]:
# ignore jupyter warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# customized functions 
def plot_cluster_frequency(adata, groupby='level_1', hue='condition', norm_dict=None, fname=None, legend_loc='best'): 
    
    plot_df = pd.DataFrame(adata.obs.groupby(groupby)[hue].value_counts().values)
    plot_df.columns = ['counts']
    plot_df[groupby] = [i[0] for i in adata.obs.groupby(groupby)[hue].value_counts().index]
    plot_df[hue] = [i[1] for i in adata.obs.groupby(groupby)[hue].value_counts().index]
    
    plot_df[hue] = plot_df[hue].astype('category')
    plot_df[hue] = plot_df[hue].cat.reorder_categories(adata.obs[hue].cat.categories)
    plot_df['counts'] = plot_df['counts'].astype(np.float)

    if norm_dict:
        plot_df['total_counts'] = plot_df[hue].values
        plot_df['total_counts'] = plot_df['total_counts'].map(norm_dict)
        plot_df['total_counts'] = plot_df['total_counts'].astype(np.float)
        plot_df['percentage'] = plot_df['counts'] / plot_df['total_counts'] * 100

    if norm_dict:
        sns.barplot(x=groupby, y='percentage', hue=hue, data=plot_df)
    else:
        sns.barplot(x=groupby, y='counts', hue=hue, data=plot_df)
    
    plt.xticks(rotation=45)
    plt.legend(loc=legend_loc)
    if fname:
        plt.savefig(os.path.join(output_path, fname))
    plt.show()

## Input

In [ ]:
# define IO path and load data object
base_path = './'
expr_path = os.path.join(base_path, 'data')

output_path = os.path.join(base_path, 'output')
if not os.path.exists(output_path): os.makedirs(output_path)
fig_path = os.path.join(base_path, "figures")
if not os.path.exists(fig_path): os.makedirs(fig_path)
    
cdata = sc.read_h5ad(os.path.join(expr_path, f'combined-raw.h5ad'))
cdata

## Preprocessing

In [ ]:
# calculate generate qc metrics
sc.pp.calculate_qc_metrics(cdata, inplace=True, percent_top=None)

In [ ]:
# reads count filtering 
sc.pp.filter_cells(cdata, min_counts=2)
sc.pp.filter_cells(cdata, min_genes=2)
cdata

In [ ]:
# check out number of cells under each condition
cdata.obs['condition'].value_counts()

In [ ]:
# preprocessing
sc.pp.normalize_total(cdata)
sc.pp.log1p(cdata)
cdata.raw = cdata
sc.pp.scale(cdata)
cdata.layers['scaled'] = cdata.X.copy()

In [ ]:
# visualize general qc metrics
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
sns.boxplot(y='total_counts', x='sample', hue='sample', data=cdata.obs, ax=axs[0])
sns.boxplot(y='n_genes_by_counts', x='sample', hue='sample', data=cdata.obs, ax=axs[1])
fig.autofmt_xdate(rotation=45)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
sns.boxplot(y='total_counts', x='condition', hue='condition', data=cdata.obs, ax=axs[0])
sns.boxplot(y='n_genes_by_counts', x='condition', hue='condition', data=cdata.obs, ax=axs[1])
plt.show()

## Kmeans clustering

### Level 1

In [ ]:
# load gene annotation table
gene_annotation = pd.read_csv(os.path.join(base_path, 'documents', 'gene_annotation.csv'))
gene_annotation.index = gene_annotation['Gene']
selected_genes = gene_annotation.loc[gene_annotation['Level_1_binary'] == True, 'Gene'].to_list()
level_1_order = gene_annotation.loc[gene_annotation['Level_1_binary'] == True, 'Level_1_annotation'].unique()

print(f"Selected genes: {len(selected_genes)}")
print(level_1_order)

In [ ]:
# check out level 1 gene markers 
gene_annotation.loc[gene_annotation['Level_1_binary'] == True, :]

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in level_1_order:
    selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_1_binary'] == True) & (gene_annotation['Level_1_annotation'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# subset gene expression profile with selected gene markers
sdata = cdata[:, selected_genes]
sdata

In [ ]:
# kmeans clustering
k = 34
kmeans = KMeans(n_clusters=k, random_state=5).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

# visualize clusters with gene markers
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)

#### Assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['level_1'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_l1 = {}

# Level_1
level_1_list = [
    'T cells', #0
    'T cells', #1
    'B cells', #2
    'NK cells', #3
    'NA', #4
    'B cells', #5 
    'Macrophages', #6
    'Macrophages', #7
    'Dendritic cells', #8
    'T cells', #9
    'Endothelial cells', #10 
    'Dendritic cells', #11
    'Macrophages', #12
    'Dendritic cells', #13
    'B cells', #14
    'Macrophages', #15
    'Endothelial cells', #16
    'Dendritic cells', #17
    'NK cells', #18
    'Macrophages', #19
    'NK cells', #20
    'B cells', #21
    'Macrophages', #22
    'B cells', #23
    'T cells', #24
    'Endothelial cells', #25
    'Macrophages', #26
    'NK cells', #27
    'Endothelial cells', #28
    'Macrophages', #29
    'Endothelial cells', #30
    'Endothelial cells', #31
    'Dendritic cells', #32
    'Dendritic cells', #33
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_l1[i] = level_1_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'level_1': transfer_dict_l1})

In [ ]:
# create variable category for level 1 annotation 
level_1_order = ['T cells', 'B cells', 'Macrophages', 'Dendritic cells', 'NK cells', 'Endothelial cells', 'NA']
sdata.obs['level_1'] = sdata.obs['level_1'].astype('category')
sdata.obs['level_1'] = sdata.obs['level_1'].cat.reorder_categories(level_1_order)

In [ ]:
# create color palette for visualization
level_1_pl = sns.color_palette(['#00A651', '#FBB040', '#92278F', '#03a5fc', '#386363', '#d12852', '#dbdbdb'])
level_1_cmap = ListedColormap(level_1_pl.as_hex())
sns.palplot(level_1_pl)
plt.xticks(range(len(level_1_order)), level_1_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(os.path.join(fig_path, 'level_2_palette.pdf'))
plt.show()

In [ ]:
# visualize cluster frequency under each condition
condition_counts =  pd.DataFrame(sdata.obs['condition'].value_counts())
condition_counts_dict = dict(zip(condition_counts.index, condition_counts['count']))

plot_cluster_frequency(sdata, groupby='level_1', hue='condition', )
plot_cluster_frequency(sdata, groupby='level_1', hue='condition', norm_dict=condition_counts_dict, )

In [ ]:
# visualize cluster frequency of each sample
sample_counts =  pd.DataFrame(sdata.obs['sample'].value_counts())
sample_counts_dict = dict(zip(sample_counts.index, sample_counts['count']))

plot_cluster_frequency(sdata, groupby='level_1', hue='sample', )
plot_cluster_frequency(sdata, groupby='level_1', hue='sample', norm_dict=sample_counts_dict, )

In [ ]:
# create gene marker dictionary for visualization
selected_gene_dict = {}
for current_type in level_1_order:
    if current_type != 'NA':
        selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_1_binary'] == True) & (gene_annotation['Level_1_annotation'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# generate gene marker heatmap for level 1 annotation
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'level_1', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'level_1', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[sdata.obs['sample'] == current_sample, :]
    
    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='level_1', data=current_obs, palette=level_1_pl, s=1, linewidth=0, ax=ax)
    plt.legend(markerscale=5)
    # plt.savefig(os.path.join(fig_path, f"sct_{current_sample}_LN.png"))
    plt.show()

In [ ]:
# map the annotation to origianl object
cdata.obs['level_1'] = 'NA'
cdata.obs['level_1'] = cdata.obs['level_1'].astype(object)
cdata.obs.loc[sdata.obs.index, 'level_1'] = sdata.obs['level_1'].values
cdata.obs['level_1'].unique()

In [ ]:
# backup data object (optional)
# from datetime import datetime
# date = datetime.today().strftime('%Y-%m-%d')
# cdata.write_h5ad(f"{output_path}/{date}-combined-celltyping-LN.h5ad")

### Level 2 

In [ ]:
# create an empty column for level 2
cdata.obs['level_2'] = cdata.obs['level_1'].values
cdata.obs['level_2'] = cdata.obs['level_2'].astype(object)

#### T cells

In [ ]:
# create gene marker dictionary for classifing t cells 
current_order = ['CD3+', 'CD3-']
selected_genes = ['Cd3e', 'Cd3d', 'Cd3g', 'Cd4', 'Cd8a']
print(f"Selected genes: {len(selected_genes)}")
print(current_order)

In [ ]:
selected_gene_dict = {'CD3+': selected_genes}
selected_gene_dict

In [ ]:
# subset data
sdata = cdata[cdata.obs['level_1'] == 'T cells', selected_genes]
sdata

In [ ]:
# kmeans elbow
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sdata.X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# kmeans
k = 10
kmeans = KMeans(n_clusters=k, random_state=10).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, vmax=2, swap_axes=True)

##### Assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['level_2'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_l2 = {}

# Level_2
level_2_list = [
    'T cells', #0
    'T cells', #1
    'CD4+ T cells', #2
    'T cells', #3
    'T cells', #4
    'NA', #5
    'CD8+ T cells', #6
    'CD4+ T cells', #7
    'T cells', #8
    'T cells', #9
    
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_l2[i] = level_2_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'level_2': transfer_dict_l2})

In [ ]:
# create variable category for level 2 t cells annotation 
current_order = ['T cells', 'CD4+ T cells', 'CD8+ T cells', 'NA']
sdata.obs['level_2'] = sdata.obs['level_2'].astype('category')
sdata.obs['level_2'] = sdata.obs['level_2'].cat.reorder_categories(current_order)

In [ ]:
# create color palette for visualization
current_pl = sns.color_palette('tab10', len(current_order))
current_cmap = ListedColormap(current_pl.as_hex())
sns.palplot(current_pl)
plt.xticks(range(len(current_order)), current_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(os.path.join(fig_path, 'level_2_palette.pdf'))
plt.show()

In [ ]:
# visualize cluster frequency under each condition
condition_counts =  pd.DataFrame(cdata.obs['condition'].value_counts())
condition_counts_dict = dict(zip(condition_counts.index, condition_counts['count']))

plot_cluster_frequency(sdata, groupby='level_2', hue='condition', )
plot_cluster_frequency(sdata, groupby='level_2', hue='condition', norm_dict=condition_counts_dict, )

In [ ]:
# generate gene marker heatmap for level 2 t cells annotation
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, figsize=(3, 15))
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True, figsize=(5, 6))
sc.pl.dotplot(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=True, cmap='Reds', swap_axes=True,)

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample), :]

    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='level_2', data=current_obs, palette='tab10', size=10, linewidth=0, ax=ax)
    plt.legend(markerscale=5)
    # plt.savefig(os.path.join(output_path, f"sct_{current_sample}.png"))
    plt.show()

In [ ]:
# map to original object
cdata.obs.loc[sdata.obs.index, 'level_2'] = sdata.obs['level_2'].values

In [ ]:
# remove NA t cells 
na_index = sdata.obs.loc[sdata.obs['level_2'] == 'NA', :].index
cdata.obs.loc[na_index, 'level_1'] = 'NA'

#### Macrophages

In [ ]:
# load gene annotation table
gene_annotation = pd.read_csv(os.path.join(base_path, 'documents', 'gene_annotation.csv'))
gene_annotation.index = gene_annotation['Gene']
selected_genes = gene_annotation.loc[gene_annotation['Level_2_binary_macrophages'] == True, 'Gene'].to_list()
current_order = gene_annotation.loc[gene_annotation['Level_2_binary_macrophages'] == True, 'Level_2_annotation_macrophages'].unique()

print(f"Selected genes: {len(selected_genes)}")
print(current_order)

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in current_order:
    selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_2_binary_macrophages'] == True) & (gene_annotation['Level_2_annotation_macrophages'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# subset data
sdata = cdata[cdata.obs['level_1'] == 'Macrophages', selected_genes]
sdata

In [ ]:
# Kmeans elbow
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sdata.X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# kmeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=5).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, vmax=2, swap_axes=True)

##### Assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['level_2'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_l2 = {}

# Level_2
level_2_list = [
    'Macrophages', #0
    'Monocytes', #1
    'Macrophages', #2
    'Activated Macrophages', #3
    'Macrophages', #4
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_l2[i] = level_2_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'level_2': transfer_dict_l2})

In [ ]:
# create variable category for level 2 macrophage annotation 
current_order = ['Macrophages', 'Activated Macrophages', 'Monocytes']
sdata.obs['level_2'] = sdata.obs['level_2'].astype('category')
sdata.obs['level_2'] = sdata.obs['level_2'].cat.reorder_categories(current_order)

In [ ]:
# create color palette for visualization
current_pl = sns.color_palette('tab10', len(current_order))
current_cmap = ListedColormap(current_pl.as_hex())
sns.palplot(current_pl)
plt.xticks(range(len(current_order)), current_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(os.path.join(fig_path, 'level_2_palette.pdf'))
plt.show()

In [ ]:
# visualize cluster frequency under each condition
condition_counts =  pd.DataFrame(cdata.obs['condition'].value_counts())
condition_counts_dict = dict(zip(condition_counts.index, condition_counts['count']))

plot_cluster_frequency(sdata, groupby='level_2', hue='condition', )
plot_cluster_frequency(sdata, groupby='level_2', hue='condition', norm_dict=condition_counts_dict, )

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in current_order:
    if current_type != 'NA':
        selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_2_binary_macrophages'] == True) & (gene_annotation['Level_2_annotation_macrophages'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# generate gene marker heatmap for level 2 macrophage annotation
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, figsize=(3, 15))
sc.pl.dotplot(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.dotplot(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='Reds', swap_axes=True)

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample), :]

    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='level_2', data=current_obs, palette='tab10', s=5, linewidth=0, ax=ax)
    plt.legend(markerscale=3)
    # plt.savefig(os.path.join(output_path, f"sct_{current_sample}.png"))
    plt.show()

In [ ]:
# map to original object
cdata.obs.loc[sdata.obs.index, 'level_2'] = sdata.obs['level_2'].values
cdata.obs['level_2'].unique()

#### Dendritic cells

In [ ]:
# load gene annotation table
gene_annotation = pd.read_csv(os.path.join(base_path, 'documents', 'gene_annotation.csv'))
gene_annotation.index = gene_annotation['Gene']
selected_genes = gene_annotation.loc[gene_annotation['Level_2_binary_dendritic_cells'] == True, 'Gene'].to_list()
current_order = gene_annotation.loc[gene_annotation['Level_2_binary_dendritic_cells'] == True, 'Level_2_annotation_dendritic_cells'].unique()

print(f"Selected genes: {len(selected_genes)}")
print(current_order)

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in current_order:
    selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_2_binary_dendritic_cells'] == True) & (gene_annotation['Level_2_annotation_dendritic_cells'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# subset data
sdata = cdata[cdata.obs['level_1'] == 'Dendritic cells', selected_genes]
sdata

In [ ]:
# Kmeans elbow
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sdata.X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# kmeans
k = 4
kmeans = KMeans(n_clusters=k, random_state=5).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, vmax=2, swap_axes=True)

##### Assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['level_2'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_l2 = {}

# Level_2
level_2_list = [
    'cDC1', #0
    'cDC2', #1
    'Other Dendritic cells', #2
    'Other Dendritic cells', #3
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_l2[i] = level_2_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'level_2': transfer_dict_l2})

In [ ]:
# create variable category for level 2 dendritic cells annotation
current_order = ['Other Dendritic cells', 'cDC1', 'cDC2']
sdata.obs['level_2'] = sdata.obs['level_2'].astype('category')
sdata.obs['level_2'] = sdata.obs['level_2'].cat.reorder_categories(current_order)

In [ ]:
# create color palette for visualization
current_pl = sns.color_palette('tab10', len(current_order))
current_cmap = ListedColormap(current_pl.as_hex())
sns.palplot(current_pl)
plt.xticks(range(len(current_order)), current_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(os.path.join(fig_path, 'level_2_palette.pdf'))
plt.show()

In [ ]:
# visualize cluster frequency under each condition
condition_counts =  pd.DataFrame(cdata.obs['condition'].value_counts())
condition_counts_dict = dict(zip(condition_counts.index, condition_counts['count']))

plot_cluster_frequency(sdata, groupby='level_2', hue='condition', )
plot_cluster_frequency(sdata, groupby='level_2', hue='condition', norm_dict=condition_counts_dict, )

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in current_order:
    if current_type != 'NA':
        selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_2_binary_dendritic_cells'] == True) & (gene_annotation['Level_2_annotation_dendritic_cells'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# generate gene marker heatmap for level 2 dendritic cell annotation
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, figsize=(3, 15))
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'level_2', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample), :]

    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='level_2', data=current_obs, palette='tab10', s=5, linewidth=0, ax=ax)
    plt.legend(markerscale=3)
    # plt.savefig(os.path.join(output_path, f"sct_{current_sample}.png"))
    plt.show()

In [ ]:
# map to original object
cdata.obs.loc[sdata.obs.index, 'level_2'] = sdata.obs['level_2'].values
cdata.obs['level_2'].unique()

### Level 3

In [ ]:
# create an empty column for level 3
cdata.obs['level_3'] = cdata.obs['level_2'].values
cdata.obs['level_3'] = cdata.obs['level_3'].astype(object)

#### CD4+/CD8+ T cells

In [ ]:
# load gene annotation table
gene_annotation = pd.read_csv(os.path.join(base_path, 'documents', 'gene_annotation.csv'))
gene_annotation.index = gene_annotation['Gene']
selected_genes = gene_annotation.loc[gene_annotation['Level_2_binary_t_cells'] == True, 'Gene'].to_list()
current_order = gene_annotation.loc[gene_annotation['Level_2_binary_t_cells'] == True, 'Level_2_annotation_t_cells'].unique()

print(f"Selected genes: {len(selected_genes)}")
print(current_order)

In [ ]:
# create gene marker dictionary
selected_gene_dict = {}
for current_type in current_order:
    selected_gene_dict[current_type] = gene_annotation.loc[(gene_annotation['Level_2_binary_t_cells'] == True) & (gene_annotation['Level_2_annotation_t_cells'] == current_type), 'Gene'].to_list()

selected_gene_dict

In [ ]:
# subset data
sdata = cdata[cdata.obs['level_2'].isin(['CD4+ T cells', 'CD8+ T cells']), selected_genes]
sdata

In [ ]:
# Kmeans elbow
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sdata.X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# # test k selection
# for i in range(50):
#     print(i)
#     kmeans = KMeans(n_clusters=25, random_state=i).fit(X_pca)
#     sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)
#     sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)

In [ ]:
# kmeans
k = 25
kmeans = KMeans(n_clusters=k, random_state=0).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, vmax=2, swap_axes=True)
sc.pl.dotplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, cmap='Reds', swap_axes=True)

##### Assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['level_3'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_l3 = {}

# Level_3
level_3_list = [
    'Naive CD8+ T cells', #0
    'Naive CD4+ T cells', #1
    'Naive CD4+ T cells', #2
    'CD8+ T cells', #3
    'CD8+ T cells', #4
    'Th17', #5 
    'PD-1+ T cells', #6
    'CD8+ T cells', #7
    'Treg', #8
    'Th1', #9
    'CD4+ T cells', #10 
    'CD4+ T cells', #11
    'Naive CD4+ T cells', #12
    'Th2', #13
    'CD8+ T cells', #14
    'Naive CD8+ T cells', #15
    'Naive CD4+ T cells', #16
    'Treg', #17
    'Th1', #18
    'CD8+ T cells', #19
    'Naive CD8+ T cells', #20
    'Treg', #21
    'PD-1+ T cells', #22
    'Naive CD4+ T cells', #23
    'CD4+ T cells', #24
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_l3[i] = level_3_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'level_3': transfer_dict_l3})

In [ ]:
# create variable category for level 3 t cell annotation 
current_order = ['CD8+ T cells', 'CD4+ T cells', 'Treg', 'Th1', 'Th2', 'Th17', 'Naive CD8+ T cells', 'Naive CD4+ T cells', 'PD-1+ T cells']
sdata.obs['level_3'] = sdata.obs['level_3'].astype('category')
sdata.obs['level_3'] = sdata.obs['level_3'].cat.reorder_categories(current_order)

In [ ]:
# create color palette for visualization
current_pl = sns.color_palette('tab20', len(current_order))
current_cmap = ListedColormap(current_pl.as_hex())
sns.palplot(current_pl)
plt.xticks(range(len(current_order)), current_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(os.path.join(fig_path, 'level_2_palette.pdf'))
plt.show()

In [ ]:
# visualize cluster frequency under each condition
condition_counts =  pd.DataFrame(cdata.obs['condition'].value_counts())
condition_counts_dict = dict(zip(condition_counts.index, condition_counts['count']))

plot_cluster_frequency(sdata, groupby='level_3', hue='condition', )
plot_cluster_frequency(sdata, groupby='level_3', hue='condition', norm_dict=condition_counts_dict, )

In [ ]:
# create gene marker dictionary
selected_gene_dict = {'T cells': ['Cd3d', 'Cd3e', 'Cd3g'],
 'CD8 T cells': ['Cd8a'],
 'CD4 T cells': ['Cd4'],
 'Treg': ['Foxp3', 'Il2ra'],
 'Th1': ['Ifng', 'Tbx21'],
 'Th2': ['Il4'],
 'Th17': ['Il17a'],
 'Naive T cells': ["Sell", "Ccr7", "Lef1"],
 'PD-1+ T cells': ['Pdcd1']}

In [ ]:
# generate gene marker heatmap for level 3 t cell annotation
sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'level_3', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, figsize=(3, 15))
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'level_3', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True, figsize=(5, 6))
sc.pl.dotplot(sdata, selected_gene_dict, groupby=f'level_3', dendrogram=False, use_raw=True, cmap='Reds', swap_axes=True)

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample), :]

    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='level_3', data=current_obs, palette='tab20', size=10, linewidth=0, ax=ax)
    # plt.savefig(os.path.join(output_path, f"sct_{current_sample}.png"))
    plt.show()

In [ ]:
# Map to original obj
cdata.obs.loc[sdata.obs.index, 'level_3'] = sdata.obs['level_3'].values
cdata.obs['level_3'].unique()

### APCs

In [ ]:
# create an empty column for APC
cdata.obs['APC'] = 'NA'

In [ ]:
# create gene marker dictionary
current_order = ['APC', 'Non-APC']
selected_genes = ['Cd40', 'Cd86', 'Ccr7', 'H2-K1']
selected_gene_dict = {'APC': selected_genes}
selected_gene_dict

In [ ]:
# subset data
sdata = cdata[cdata.obs['level_1'].isin(['Dendritic cells', 'B cells', 'Macrophages']), selected_genes]
sdata

In [ ]:
# Kmeans elbow
distorsions = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sdata.X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# kmeans
k = 6
kmeans = KMeans(n_clusters=k, random_state=10).fit(sdata.X)
sdata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)

sc.pl.heatmap(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=False, cmap='bwr', vmin=-4, vmax=4, swap_axes=True)
sc.pl.matrixplot(sdata, selected_gene_dict, groupby=f'kmeans{k}', dendrogram=False, use_raw=True, vmax=2, swap_axes=True)

##### assign cell types

In [ ]:
# create backup for kmeans label
sdata.obs['APC'] = sdata.obs[f'kmeans{k}'].values

In [ ]:
# change cluster label to cell type label
transfer_dict_apc = {}

# apc
apc_list = [
    'Non-APC', #0
    'APC', #1
    'APC', #2
    'APC', #3
    'APC', #4
    'APC', #5
]

# construct transfer dict
for i in sorted(sdata.obs[f'kmeans{k}'].unique()):
    transfer_dict_apc[i] = apc_list[int(i)]

In [ ]:
# assign cell type to sdata
sdata.obs = sdata.obs.replace({'APC': transfer_dict_apc})

In [ ]:
# get APC category color
current_order = ['APC', 'Non-APC']
sdata.obs['APC'] = sdata.obs['APC'].astype('category')
sdata.obs['APC'] = sdata.obs['APC'].cat.reorder_categories(current_order)

In [ ]:
# create color palette
current_pl = sns.color_palette('tab10', len(current_order))
current_cmap = ListedColormap(current_pl.as_hex())
sns.palplot(current_pl)
plt.xticks(range(len(current_order)), current_order, size=10, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# compare with level2 annotation
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(pd.crosstab(sdata.obs['level_2'], sdata.obs['APC']), annot=True, fmt='g')

In [ ]:
# visualize spatial organization of each sample 
for current_sample in sdata.obs['sample'].cat.categories:
    print(current_sample)
    current_complete_obs = cdata.obs.loc[cdata.obs['sample'] == current_sample, :]
    current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample), :]
    # current_obs = sdata.obs.loc[(sdata.obs['sample'] == current_sample) & (sdata.obs['level_2'] != "T cells"), :]

    fig_size = np.array([current_complete_obs['global_x'].max(), current_complete_obs['global_y'].max()]) / 1000
    fig, ax = plt.subplots(figsize=fig_size)
    sns.scatterplot(x='global_x', y='global_y', data=current_complete_obs, color='#dbdbdb', s=1, linewidth=0, ax=ax)
    sns.scatterplot(x='global_x', y='global_y', hue='APC', data=current_obs, palette='tab10', size=10, linewidth=0, ax=ax)
    # plt.savefig(os.path.join(output_path, f"sct_{current_sample}.png"))
    plt.show()

In [ ]:
# Map to original obj
cdata.obs.loc[sdata.obs.index, 'APC'] = sdata.obs['APC'].values

## Output

In [ ]:
# output data object with assigned cell types 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
cdata.write_h5ad(f"{output_path}/{date}-combined-celltyping-LN.h5ad")